# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 3. Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

1. Создать Dataset для загрузки данных
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHES = 15
LR = 0.1

In [3]:
class CHDataset(Dataset):
    def __init__(self, *init_datasets):
        assert all(init_datasets[0].size(0) == init_dataset.size(0) for init_dataset in init_datasets), "Не соотвутствует размерность среди dataset"
        self._base_datasets = init_datasets
  
    def __len__(self):
        return self._base_datasets[0].size(0)
  
    def __getitem__(self, idx):
        return tuple(base_dataset[idx] for base_dataset in self._base_datasets)

In [4]:
class CHNet(nn.Module):
    def __init__(self) -> None:
        super(CHNet, self).__init__()
        self.block_1 = nn.Sequential(
            nn.Linear(in_features=8, out_features=100, bias=True),
            nn.Dropout(0.1),
            nn.BatchNorm1d(100),
            nn.ReLU())
        self.block_2 = nn.Sequential(
            nn.Linear(in_features=100, out_features=100, bias=True),
            nn.Dropout(0.2),
            nn.BatchNorm1d(100),
            nn.ReLU())
        self.block_3 = nn.Sequential(
            nn.Linear(in_features=100, out_features=60, bias=True),
            nn.Dropout(0.2),
            nn.BatchNorm1d(60),
            nn.ReLU())
        self.block_4 = nn.Sequential(
            nn.Linear(in_features=60, out_features=30),
            nn.Dropout(0.2),
            nn.BatchNorm1d(30),
            nn.ReLU())
        self.predict = nn.Sequential(
            nn.Linear(in_features=30, out_features=1, bias=True),
            nn.BatchNorm1d(1),
            nn.ReLU())
  
    def forward(self, inp):
        out = self.block_1(inp)
        out = self.block_2(out)
        out = self.block_3(out)
        out = self.block_4(out)
        out = self.predict(out)
        return out[:, 0]

In [5]:
def train_loop(train_loader, test_loader, net, optimizer):
    loss_fn = nn.MSELoss()
    best_acc = {'train': None, 'test': None}
    net.train()
    for epoch in range(EPOCHES):
        running_loss, running_items, running_right = 0.0, 0.0, 0.0
        for i, (inputs, labels) in enumerate(train_loader):

            outputs = net(inputs)
            loss = loss_fn(outputs, labels)

            # обнуляем градиент
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            running_items += len(labels)

            # выводим статистику о процессе обучения
            if i % 150 == 0 or (i + 1) == len(train_loader):    # печатаем каждые 150 mini-batches
                net.eval()

                test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
                for y, (out_test, lbl_test) in enumerate(test_loader):
                    test_outputs = net(out_test)
                    test_loss += loss_fn(test_outputs, lbl_test)
                    test_running_total += len(lbl_test)

                res_loss_train = running_loss / running_items
                res_loss_test = test_loss / test_running_total

                if best_acc['train'] is None or res_loss_train < best_acc['train']:
                    best_acc['train'] = res_loss_train

                if best_acc['test'] is None or res_loss_test < best_acc['test']:
                    best_acc['test'] = res_loss_train

                print(f'Epoch [{epoch + 1}/{EPOCHES}]. ' \
                      f'Step [{i + 1}/{len(train_loader)}]. ' \
                      f'Loss: {res_loss_train:.3f}. '\
                      f'Test acc: {res_loss_test:.3f}.')

                running_loss, running_items = 0.0, 0.0
                net.train()
    print(f"Best acc train: {best_acc['train']:.3f}. Best acc test: {best_acc['test']:.3f}")
    print('Training is finished!')
    return (best_acc['train'], best_acc['test'])

In [6]:
california_housing = fetch_california_housing()
# Разделим на тестовые и тренеровочные данные
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target, test_size=0.25, random_state=13)

In [7]:
# Нормализуем данные и подготовим их для дальнейшего использования в нашем dstaset
scale = StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [8]:
train_xt = torch.from_numpy(X_train_s.astype(np.float32)).to(DEVICE)
train_yt = torch.from_numpy(y_train.astype(np.float32)).to(DEVICE)

test_xt = torch.from_numpy(X_test_s.astype(np.float32)).to(DEVICE)
test_yt = torch.from_numpy(y_test.astype(np.float32)).to(DEVICE)

In [9]:
train_dataset = CHDataset(train_xt, train_yt)
test_dataset = CHDataset(test_xt, test_yt)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=False)

In [11]:
result = {}

# Adam

In [12]:
net = CHNet().to(DEVICE)
optimizer = Adam(net.parameters(), lr=LR)

In [13]:
%%time
result['adam'] = train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/15]. Step [1/242]. Loss: 0.080. Test acc: 0.089.
Epoch [1/15]. Step [151/242]. Loss: 0.012. Test acc: 0.008.
Epoch [1/15]. Step [242/242]. Loss: 0.008. Test acc: 0.008.
Epoch [2/15]. Step [1/242]. Loss: 0.004. Test acc: 0.008.
Epoch [2/15]. Step [151/242]. Loss: 0.007. Test acc: 0.007.
Epoch [2/15]. Step [242/242]. Loss: 0.007. Test acc: 0.007.
Epoch [3/15]. Step [1/242]. Loss: 0.008. Test acc: 0.007.
Epoch [3/15]. Step [151/242]. Loss: 0.007. Test acc: 0.012.
Epoch [3/15]. Step [242/242]. Loss: 0.007. Test acc: 0.009.
Epoch [4/15]. Step [1/242]. Loss: 0.007. Test acc: 0.008.
Epoch [4/15]. Step [151/242]. Loss: 0.007. Test acc: 0.007.
Epoch [4/15]. Step [242/242]. Loss: 0.007. Test acc: 0.014.
Epoch [5/15]. Step [1/242]. Loss: 0.007. Test acc: 0.014.
Epoch [5/15]. Step [151/242]. Loss: 0.007. Test acc: 0.010.
Epoch [5/15]. Step [242/242]. Loss: 0.006. Test acc: 0.009.
Epoch [6/15]. Step [1/242]. Loss: 0.007. Test acc: 0.011.
Epoch [6/15]. Step [151/242]. Loss: 0.007. Test acc:

# RMSProp

In [14]:
net = CHNet().to(DEVICE)
optimizer = RMSprop(net.parameters(), lr=LR)

In [15]:
%%time
result['rms'] = train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/15]. Step [1/242]. Loss: 0.075. Test acc: 0.089.
Epoch [1/15]. Step [151/242]. Loss: 0.009. Test acc: 0.006.
Epoch [1/15]. Step [242/242]. Loss: 0.007. Test acc: 0.011.
Epoch [2/15]. Step [1/242]. Loss: 0.005. Test acc: 0.014.
Epoch [2/15]. Step [151/242]. Loss: 0.007. Test acc: 0.013.
Epoch [2/15]. Step [242/242]. Loss: 0.007. Test acc: 0.006.
Epoch [3/15]. Step [1/242]. Loss: 0.011. Test acc: 0.006.
Epoch [3/15]. Step [151/242]. Loss: 0.007. Test acc: 0.007.
Epoch [3/15]. Step [242/242]. Loss: 0.007. Test acc: 0.013.
Epoch [4/15]. Step [1/242]. Loss: 0.007. Test acc: 0.013.
Epoch [4/15]. Step [151/242]. Loss: 0.007. Test acc: 0.009.
Epoch [4/15]. Step [242/242]. Loss: 0.007. Test acc: 0.007.
Epoch [5/15]. Step [1/242]. Loss: 0.005. Test acc: 0.006.
Epoch [5/15]. Step [151/242]. Loss: 0.007. Test acc: 0.006.
Epoch [5/15]. Step [242/242]. Loss: 0.006. Test acc: 0.012.
Epoch [6/15]. Step [1/242]. Loss: 0.005. Test acc: 0.011.
Epoch [6/15]. Step [151/242]. Loss: 0.006. Test acc:

# SGD

In [16]:
net = CHNet().to(DEVICE)
optimizer = SGD(net.parameters(), lr=LR)

In [17]:
%%time
result['sgd'] = train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/15]. Step [1/242]. Loss: 0.050. Test acc: 0.078.
Epoch [1/15]. Step [151/242]. Loss: 0.011. Test acc: 0.008.
Epoch [1/15]. Step [242/242]. Loss: 0.008. Test acc: 0.007.
Epoch [2/15]. Step [1/242]. Loss: 0.006. Test acc: 0.007.
Epoch [2/15]. Step [151/242]. Loss: 0.008. Test acc: 0.008.
Epoch [2/15]. Step [242/242]. Loss: 0.007. Test acc: 0.006.
Epoch [3/15]. Step [1/242]. Loss: 0.010. Test acc: 0.007.
Epoch [3/15]. Step [151/242]. Loss: 0.007. Test acc: 0.007.
Epoch [3/15]. Step [242/242]. Loss: 0.007. Test acc: 0.007.
Epoch [4/15]. Step [1/242]. Loss: 0.007. Test acc: 0.007.
Epoch [4/15]. Step [151/242]. Loss: 0.007. Test acc: 0.007.
Epoch [4/15]. Step [242/242]. Loss: 0.007. Test acc: 0.006.
Epoch [5/15]. Step [1/242]. Loss: 0.007. Test acc: 0.006.
Epoch [5/15]. Step [151/242]. Loss: 0.007. Test acc: 0.006.
Epoch [5/15]. Step [242/242]. Loss: 0.007. Test acc: 0.009.
Epoch [6/15]. Step [1/242]. Loss: 0.004. Test acc: 0.009.
Epoch [6/15]. Step [151/242]. Loss: 0.007. Test acc:

# SGD + Momentum

In [18]:
net = CHNet().to(DEVICE)
optimizer = SGD(net.parameters(), lr=LR, momentum=0.8)

In [19]:
%%time
result['sgdm'] = train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/15]. Step [1/242]. Loss: 0.102. Test acc: 0.089.
Epoch [1/15]. Step [151/242]. Loss: 0.011. Test acc: 0.008.
Epoch [1/15]. Step [242/242]. Loss: 0.008. Test acc: 0.007.
Epoch [2/15]. Step [1/242]. Loss: 0.007. Test acc: 0.007.
Epoch [2/15]. Step [151/242]. Loss: 0.008. Test acc: 0.007.
Epoch [2/15]. Step [242/242]. Loss: 0.007. Test acc: 0.010.
Epoch [3/15]. Step [1/242]. Loss: 0.007. Test acc: 0.009.
Epoch [3/15]. Step [151/242]. Loss: 0.007. Test acc: 0.009.
Epoch [3/15]. Step [242/242]. Loss: 0.007. Test acc: 0.007.
Epoch [4/15]. Step [1/242]. Loss: 0.009. Test acc: 0.007.
Epoch [4/15]. Step [151/242]. Loss: 0.007. Test acc: 0.007.
Epoch [4/15]. Step [242/242]. Loss: 0.007. Test acc: 0.007.
Epoch [5/15]. Step [1/242]. Loss: 0.005. Test acc: 0.007.
Epoch [5/15]. Step [151/242]. Loss: 0.007. Test acc: 0.006.
Epoch [5/15]. Step [242/242]. Loss: 0.007. Test acc: 0.011.
Epoch [6/15]. Step [1/242]. Loss: 0.005. Test acc: 0.011.
Epoch [6/15]. Step [151/242]. Loss: 0.007. Test acc:

In [20]:
print(f"Best acc train: {result['adam'][0]:.3f}. Best acc test: {result['adam'][1]:.3f}")
print(f"Best acc train: {result['rms'][0]:.3f}. Best acc test: {result['rms'][1]:.3f}")
print(f"Best acc train: {result['sgd'][0]:.3f}. Best acc test: {result['sgd'][1]:.3f}")
print(f"Best acc train: {result['sgdm'][0]:.3f}. Best acc test: {result['sgdm'][1]:.3f}")

Best acc train: 0.004. Best acc test: 0.005
Best acc train: 0.005. Best acc test: 0.005
Best acc train: 0.003. Best acc test: 0.006
Best acc train: 0.004. Best acc test: 0.004
